In [166]:
# importing all necessary libs
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objs as go
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")


In [9]:
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

# Data reading
df = pd.read_excel(parent_dir + '/data/globalterrorismdb_0221dist.xlsx')

_df = df[['eventid','iyear','imonth','iday','country','country_txt',
         'region','region_txt', 'provstate', 'city','latitude','longitude',
         'gname','nkill', 'nwound']]

In [26]:
# Geojson for polygons of map
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
                          
countries_df = pd.json_normalize(countries,  record_path =['features'])

In [99]:
# Updating the countries to be able to merge -- TODO: !!!!
_df.loc[_df['country_txt'] == 'Bahamas','country_txt']  = 'The Bahamas'
_df.loc[_df['country_txt'] == 'United States','country_txt'] = 'United States of America'
_df.loc[_df['country_txt'] == 'Czechoslovakia','country_txt'] = 'Czech Republic'
_df.loc[_df['country_txt'] == 'East Germany (GDR)','country_txt']  = 'Germany'
_df.loc[_df['country_txt'] == 'West Germany (FRG)','country_txt']  = 'Germany'
_df.loc[_df['country_txt'] == 'Bosnia-Herzegovina','country_txt'] = 'Bosnia and Herzegovina'
_df.loc[_df['country_txt'] == 'West Bank and Gaza Strip','country_txt'] = 'West Bank'

In [103]:
# Merging locations and main df
df_merged = pd.merge(_df, countries_df , left_on = 'country_txt', right_on = 'properties.name', how = 'left' )

# Determine ig organized or unorganized
df_merged['isOrganized'] = ['unorganized' if i == 'Unknown' else 'organized' for i in df_merged['gname']]

In [189]:
# We need safety index calculation before groupby
df_map = df_merged.groupby(by=['country','country_txt','id']).agg({'nkill':np.mean}).reset_index()

def update_map():
    df_map['hover_text'] = 'Country: ' + df_map["country_txt"].astype(str) + "\n" + 'Safety Index: ' + df_map['nkill'].round(decimals= 2).astype(str)
    
    trace = go.Choropleth(locations=df_map['id'],
                          z=df_map['nkill'],
                          text=df_map['hover_text'], 
                          hoverinfo='text' ,
                          marker_line_color='white',
                          colorscale="Reds",
                          reversescale = True,
                          marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 20,"len": 0.7, "x": 0.9, "y": 0.7,
                                   'title': {"text": 'Safety Index', "side": "bottom"},
                                 }
                         )   
    return {"data": [trace],
            "layout": go.Layout(height=600,geo={'showframe': False,
                                                'showcoastlines': False,
                                                'projection': {'type': "miller"}})}



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Dashboard'


app.layout = html.Div([
    html.Div([html.H1("Global Terrorism Dashboard")],
                 style={'textAlign': "center", "padding-bottom": "30"}, className="six-column"),
    html.Div(dcc.Graph(id="world-map", figure = update_map()), className="twelve columns"),
    html.H2(id = 'country-name',  children=["init"] , style={'textAlign': "center", "padding-bottom": "30"} ),
    html.Div(dcc.Graph(id="incidents"), className="five columns"),
    html.Div(dcc.Graph(id="fatalities"), className="five columns")
])


##### Figure Updates with CallBacks ##### 
@app.callback(
    dash.dependencies.Output('country-name', 'children'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_title(clickData):
    text = ''
    if clickData is not None:
        selected_country = clickData['points'][0]['location']
        text = str(selected_country)  + '-' + str(clickData['points'][0]['text'])
    return text



@app.callback(
    dash.dependencies.Output('incidents', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_incidents(clickData):
    df_agg = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                        'country_txt','id', 'isOrganized']).agg({
                                                                                        'eventid':'count'}).reset_index()
    fig = px.line(df_agg, x="iyear", y="eventid", color='isOrganized',
                  labels={
                     "iyear": "<b>Year</b>",
                     "eventid": "<b>Number of Incidents</b>",
                     "isOrganized": " "
                 }, title="<b>Number of Incidents by Years</b>")
    
    return fig

@app.callback(
    dash.dependencies.Output('fatalities', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_fatalities(clickData):
    df_agg = ''
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        
        
        
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                        'country_txt','id']).agg({
                                                                                        'nkill':np.sum,
                                                                                        'nwound':np.sum  }).reset_index()
    # Add traces
    fig.add_trace(
        go.Scatter(x=df_agg['iyear'], y=df_agg['nwound'], name="Injuries", line=dict(color="#D4AC0D")),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df_agg['iyear'], y=df_agg['nkill'], name="Fatalities", line=dict(color="#943126")),
        secondary_y=True,
    )     
    
    # Add figure title
    fig.update_layout(
        title_text="<b>Number of Injuries and Fatalities by Years</b>"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Year</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Injuries</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Fatalities</b> ", secondary_y=True)

    
    return fig
        

    
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun